## Used for selecting the 6 best features per cluster
* We're using mean squared error of each variable vs. the ALSFRS_score, and take the best 6. 

In [8]:
%matplotlib inline

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from IPython.display import display

from modeling_funcs import *


In [9]:
vectorized_data = pd.read_csv('../train_data_vectorized.csv', sep='|', index_col=0)
slope = pd.read_csv('../train_slope.csv', sep = '|', index_col=0)
clusters = pd.read_csv('../train_kmeans_clusters.csv', sep = '|', index_col=0)
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )

X = vectorized_data.join(clusters)
Y = slope.join(clusters)
display(X.head())
display(Y.head())

,family_ALS_hist_last,Hematocrit_pct_diff,Hematocrit_last,Hematocrit_mean_slope,Hematocrit_mean,Prothrombin Time (clotting)_pct_diff,Prothrombin Time (clotting)_last,Prothrombin Time (clotting)_mean_slope,Prothrombin Time (clotting)_mean,weight_pct_diff,...,Protein_mean,Uric Acid_pct_diff,Uric Acid_last,Uric Acid_mean_slope,Uric Acid_mean,ALT(SGPT)_pct_diff,ALT(SGPT)_last,ALT(SGPT)_mean_slope,ALT(SGPT)_mean,cluster
SubjectID,,,,,,,,,,,,,,,,,,,,,
533,0,0.458821,0.341017,0.328455,0.229323,0,-0.620585,0,-0.620628,-0.232289,...,-0.767815,0,0,0,0,0.218664,-0.527906,0.050588,-0.738027,0
649,0,0.345206,0.070538,0.333508,-0.010159,0,0.000000,0,0.000000,0.218932,...,-0.381805,0,0,0,0,0.636520,-0.480243,-0.077370,-0.953654,1
1234,0,-0.620018,0.406587,-0.193480,0.623643,0,0.000000,0,0.000000,0.340632,...,0.518886,0,0,0,0,-0.139532,-0.527906,-0.103453,-0.643690,2
2492,0,-0.575963,0.185287,-0.230124,0.245839,0,0.000000,0,0.000000,-0.047759,...,0.000000,0,0,0,0,-0.097226,1.283284,0.044010,1.418247,1
2956,0,-0.210464,0.250857,0.115924,0.324290,0,0.000000,0,0.000000,-0.149486,...,0.261546,0,0,0,0,-0.029158,-0.337255,-0.057131,-0.508923,0


,ALSFRS_slope,cluster
SubjectID,,
533,-0.965608,0
649,-0.921717,1
1234,-0.914787,2
2492,-0.598361,1
2956,-0.444039,0


In [10]:
best_features_per_cluster = get_best_features_per_cluster(X, Y, all_feature_metadata)
best_features_per_cluster

{0: ['ALT(SGPT)', 'Age', 'Gender', 'Neutrophils', 'Protein', 'Race'],
 1: ['Absolute Band Neutrophil Count',
  'Absolute Monocyte Count',
  'Absolute Neutrophil Count',
  'Age',
  'Albumin',
  'Bilirubin (Total)'],
 2: ['BMI',
  'Gender',
  'Neutrophils',
  'Potassium',
  'if_use_Riluzole',
  'respiratory_rate']}

In [11]:
with open("../best_features_per_cluster.pickle", "wb") as output_file:
    pickle.dump(best_features_per_cluster, output_file)


#Apply the selector 
leave only the best features per cluster

In [12]:
for t in ["train", "test"]:
    print t
    df = pd.read_csv('../' + t + '_data.csv', sep = '|', index_col="SubjectID", dtype='unicode')
    print "df", df.shape
    clusters = pd.read_csv('../' + t + '_kmeans_clusters.csv', sep = '|', index_col="SubjectID")
    print "clusters", clusters.shape
    buf = filter_only_selected_features(df, clusters, best_features_per_cluster)
    with open('../' + t + '_data_selected.csv','w') as f:
        f.write(buf)


train
df (1138647, 5)
clusters (1777, 1)
test
df (126664, 5)
clusters (600, 1)
